In [1]:
from pycollisiondb.pycollisiondb import PyCollision
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from pyvalem.formula import Formula
%matplotlib notebook

In [2]:
# Search for all cross section datasets involving the product W+60 in state n=25.
query = {'products': ['W+60 n=50'],
         'reactants': ['H 1s', 'W+61'],
         'data_type': 'cross section'}
pycoll = PyCollision.get_datasets(query=query)
print(pycoll)

<PyCollision object (40 datasets)>


In [3]:
len(pycoll.datasets)

40

In [4]:
pycoll.get_distinct_rps()
#pycoll.reactant_species, pycoll.product_species

In [5]:
pycoll.datasets_compatible()

(True,
 ('cross section',
  'target',
  [{'name': 'E', 'units': 'eV.u-1'}, {'name': 'sigma', 'units': 'cm2'}]))

In [6]:
def aggregate(pycoll):
    
    compatible, (data_type, frame, columns) = pycoll.datasets_compatible(None, raise_exception=False)
    if not compatible:
        print('Cannot aggregate incompatible datasets')
        return
    print(data_type, frame, columns)
    
    d = {}
    nrps = None
    for pk, ds in pycoll.datasets.items():
        r_rps, p_rps = [], []
        for _, reactant_rp in ds.reaction.reactants:
            reactant_rp = (reactant_rp)
            r_rps.append(reactant_rp)
        for _, product_rp in ds.reaction.products:
            product_rp = (product_rp)
            p_rps.append(product_rp)
        if not nrps:
            nrps = len(r_rps) + len(p_rps)
        else:
            assert nrps == len(r_rps) + len(p_rps)
        rps = tuple(r_rps + p_rps)
        d[rps] = pd.Series(ds.y, index=ds.x)
        
    df = pd.DataFrame(d)
    
    rp_counts = [len(set(e[i] for e in d.keys())) for i in range(nrps)]
    rp_counts = np.array(rp_counts, dtype=np.int8)

    # Only allow one RP position to be greater than 1
    assert sum(rp_counts==1) == nrps - 1

    drop_level_idx = np.where(rp_counts == 1)[0]
    df = df.droplevel(list(drop_level_idx), axis=1)

    wildcarded_rps = []
    j = 0
    for i, rp  in enumerate(r_rps):
        if i in drop_level_idx:
            wildcarded_rps.append(str(rp))
        else:
            wildcarded_rps.append(str(rp.formula) + ' (*)')
    wildcarded_reactants = ' + '.join(wildcarded_rps)

    wildcarded_rps = []
    for i, rp  in enumerate(p_rps, start=len(r_rps)):
        if i in drop_level_idx:
            wildcarded_rps.append(str(rp))
        else:
            wildcarded_rps.append(str(rp.formula) + ' (*)')
    wildcarded_products = ' + '.join(wildcarded_rps)

    wildcarded_rxn = ' -> '.join([wildcarded_reactants, wildcarded_products])

    print(wildcarded_rxn)
    print(df)
    return
    
aggregate(pycoll)

cross section target [{'name': 'E', 'units': 'eV.u-1'}, {'name': 'sigma', 'units': 'cm2'}]
W+61 + H 1s -> W+60 (*) + H+
           W+60 n=50;l=0  W+60 n=50;l=10  W+60 n=50;l=11  W+60 n=50;l=12  \
100000.0    0.000000e+00    2.082000e-20    3.449000e-20    3.747000e-20   
500000.0    3.660000e-21    1.075000e-19    1.163000e-19    1.251000e-19   
850000.0    3.169000e-22    1.096000e-20    1.132000e-20    1.215000e-20   
1000000.0   4.311000e-23    4.756000e-21    5.805000e-21    6.050000e-21   

           W+60 n=50;l=13  W+60 n=50;l=14  W+60 n=50;l=15  W+60 n=50;l=16  \
100000.0     5.293000e-20    9.754000e-20    1.190000e-19    1.647000e-19   
500000.0     1.435000e-19    1.518000e-19    1.687000e-19    1.892000e-19   
850000.0     1.547000e-20    1.591000e-20    1.709000e-20    2.005000e-20   
1000000.0    6.625000e-21    7.760000e-21    8.694000e-21    1.029000e-20   

           W+60 n=50;l=17  W+60 n=50;l=18  ...  W+60 n=50;l=37  \
100000.0     2.070000e-19    2.694000e-19  ... 

In [9]:
rp_counts = [len(set(e[i] for e in d.keys())) for i in range(nrps)]
rp_counts = np.array(rp_counts, dtype=np.int8)

# Only allow one RP position to be greater than 1
assert sum(rp_counts==1) == nrps - 1

drop_level_idx = np.where(rp_counts == 1)[0]
df = df.droplevel(list(drop_level_idx), axis=1)

wildcarded_rps = []
j = 0
for i, rp  in enumerate(r_rps):
    if i in drop_level_idx:
        wildcarded_rps.append(str(rp))
    else:
        wildcarded_rps.append(str(rp.formula) + ' (*)')
wildcarded_reactants = ' + '.join(wildcarded_rps)

wildcarded_rps = []
for i, rp  in enumerate(p_rps, start=len(r_rps)):
    if i in drop_level_idx:
        wildcarded_rps.append(str(rp))
    else:
        wildcarded_rps.append(str(rp.formula) + ' (*)')
wildcarded_products = ' + '.join(wildcarded_rps)

wildcarded_rxn = ' -> '.join([wildcarded_reactants, wildcarded_products])

print(wildcarded_rxn)
df

W+61 + H 1s -> W+60 (*) + H+


,W+60 n=50;l=0,W+60 n=50;l=10,W+60 n=50;l=11,W+60 n=50;l=12,W+60 n=50;l=13,W+60 n=50;l=14,W+60 n=50;l=15,W+60 n=50;l=16,W+60 n=50;l=17,W+60 n=50;l=18,...,W+60 n=50;l=37,W+60 n=50;l=38,W+60 n=50;l=3,W+60 n=50;l=4,W+60 n=50;l=5,W+60 n=50;l=6,W+60 n=50;l=7,W+60 n=50;l=8,W+60 n=50;l=9,W+60 n=50
100000.0,0.000000e+00,2.082000e-20,3.449000e-20,3.747000e-20,5.293000e-20,9.754000e-20,1.190000e-19,1.647000e-19,2.070000e-19,2.694000e-19,...,6.393000e-19,5.947000e-22,1.190000e-21,0.000000e+00,1.190000e-21,4.758000e-21,8.326000e-21,7.732000e-21,1.011000e-20,4.319000e-17
500000.0,3.660000e-21,1.075000e-19,1.163000e-19,1.251000e-19,1.435000e-19,1.518000e-19,1.687000e-19,1.892000e-19,2.028000e-19,2.281000e-19,...,0.000000e+00,0.000000e+00,3.099000e-20,3.819000e-20,4.856000e-20,5.759000e-20,7.558000e-20,8.351000e-20,8.846000e-20,6.645000e-18
850000.0,3.169000e-22,1.096000e-20,1.132000e-20,1.215000e-20,1.547000e-20,1.591000e-20,1.709000e-20,2.005000e-20,2.280000e-20,2.664000e-20,...,0.000000e+00,0.000000e+00,2.725000e-21,3.359000e-21,4.669000e-21,5.155000e-21,7.268000e-21,7.162000e-21,8.430000e-21,4.083000e-19
1000000.0,4.311000e-23,4.756000e-21,5.805000e-21,6.050000e-21,6.625000e-21,7.760000e-21,8.694000e-21,1.029000e-20,1.244000e-20,1.302000e-20,...,0.000000e+00,0.000000e+00,1.308000e-21,1.710000e-21,2.271000e-21,2.860000e-21,3.032000e-21,3.679000e-21,4.239000e-21,1.712000e-19


In [34]:
df.droplevel([0,1,2], axis=1)

,W+60 n=50;l=0,W+60 n=50;l=10,W+60 n=50;l=11,W+60 n=50;l=12,W+60 n=50;l=13,W+60 n=50;l=14,W+60 n=50;l=15,W+60 n=50;l=16,W+60 n=50;l=17,W+60 n=50;l=18,...,W+60 n=50;l=37,W+60 n=50;l=38,W+60 n=50;l=3,W+60 n=50;l=4,W+60 n=50;l=5,W+60 n=50;l=6,W+60 n=50;l=7,W+60 n=50;l=8,W+60 n=50;l=9,W+60 n=50
100000.0,0.000000e+00,2.082000e-20,3.449000e-20,3.747000e-20,5.293000e-20,9.754000e-20,1.190000e-19,1.647000e-19,2.070000e-19,2.694000e-19,...,6.393000e-19,5.947000e-22,1.190000e-21,0.000000e+00,1.190000e-21,4.758000e-21,8.326000e-21,7.732000e-21,1.011000e-20,4.319000e-17
500000.0,3.660000e-21,1.075000e-19,1.163000e-19,1.251000e-19,1.435000e-19,1.518000e-19,1.687000e-19,1.892000e-19,2.028000e-19,2.281000e-19,...,0.000000e+00,0.000000e+00,3.099000e-20,3.819000e-20,4.856000e-20,5.759000e-20,7.558000e-20,8.351000e-20,8.846000e-20,6.645000e-18
850000.0,3.169000e-22,1.096000e-20,1.132000e-20,1.215000e-20,1.547000e-20,1.591000e-20,1.709000e-20,2.005000e-20,2.280000e-20,2.664000e-20,...,0.000000e+00,0.000000e+00,2.725000e-21,3.359000e-21,4.669000e-21,5.155000e-21,7.268000e-21,7.162000e-21,8.430000e-21,4.083000e-19
1000000.0,4.311000e-23,4.756000e-21,5.805000e-21,6.050000e-21,6.625000e-21,7.760000e-21,8.694000e-21,1.029000e-20,1.244000e-20,1.302000e-20,...,0.000000e+00,0.000000e+00,1.308000e-21,1.710000e-21,2.271000e-21,2.860000e-21,3.032000e-21,3.679000e-21,4.239000e-21,1.712000e-19


In [8]:
import itertools

In [9]:
itertools.count?

In [190]:
def aggregate(pycoll):
    rootd = {}#defaultdict(dict)
    for ds in pycoll.datasets.values():
        nd = rootd
        for _, reactant_rp in ds.reaction.reactants:
            reactant_rp = str(reactant_rp)
            if reactant_rp in nd.keys():
                nd = nd[reactant_rp]
            else:
                nd[reactant_rp] = {}
                nd = nd[reactant_rp]
        for _, product_rp in ds.reaction.products:
            product_rp = str(product_rp)
            if product_rp in nd.keys():
                nd = nd[product_rp]
            else:
                nd[product_rp] = {}
                if product_rp != 'H+':
                    nd = nd[product_rp]
        #nd[product_rp] = ds
        nd[product_rp] = pd.Series(ds.y, index=ds.x)
    return rootd
d = aggregate(pycoll)
d#['W+61']['H 1s']['W+60 n=50;l=0']['H+']
df = pd.from_dict(d)
df

AttributeError: module 'pandas' has no attribute 'from_dict'

In [164]:
pycoll.datasets[78341]

D78341: W+61 + H 1s → W+60 n=50;l=10 + H+

In [38]:
dset1 = pycoll.datasets[78341]
ds1 = pd.Series(dset1.y, index=dset1.x)
ds1.index.name = 'E'
ds1.name = 'n=50; l=1'

dset2 = pycoll.datasets[78340]
ds2 = pd.Series(dset2.y, index=dset2.x)
ds2.index.name = 'E'
ds2.name = 'n=50; l=0'

ds2

E
100000.0     0.000000e+00
500000.0     3.660000e-21
850000.0     3.169000e-22
1000000.0    4.311000e-23
Name: n=50; l=0, dtype: float64

In [41]:
df = pd.DataFrame((ds1, ds2)).T

In [42]:
df.sort_index(axis=1)

,n=50; l=0,n=50; l=1
E,,
100000.0,0.000000e+00,2.082000e-20
500000.0,3.660000e-21,1.075000e-19
850000.0,3.169000e-22,1.096000e-20
1000000.0,4.311000e-23,4.756000e-21


In [28]:
df

E
0.000000e+00     100000.0
3.660000e-21     500000.0
3.169000e-22     850000.0
4.311000e-23    1000000.0
Name: n=50; l=0, dtype: float64

In [43]:
pycoll.reactant_species

defaultdict(set, {H: {(1s,)}, W+61: {()}})

In [ ]:
def validate_reaction_state_counts(pycoll):
    

In [49]:
for species, states in pycoll.reactant_species.items():
    print(species, states)
#pycoll.reactant_species['H']

H {(1s,)}
W+61 {()}


In [50]:
for species, states in pycoll.product_species.items():
    print(species, states)

H+ {()}
W+60 {(n=50, l=35), (n=50, l=24), (n=50, l=2), (n=50, l=11), (n=50, l=32), (n=50,), (n=50, l=17), (n=50, l=36), (n=50, l=33), (n=50, l=29), (n=50, l=0), (n=50, l=25), (n=50, l=21), (n=50, l=15), (n=50, l=5), (n=50, l=16), (n=50, l=14), (n=50, l=18), (n=50, l=20), (n=50, l=31), (n=50, l=6), (n=50, l=9), (n=50, l=13), (n=50, l=3), (n=50, l=30), (n=50, l=4), (n=50, l=7), (n=50, l=34), (n=50, l=8), (n=50, l=19), (n=50, l=37), (n=50, l=38), (n=50, l=10), (n=50, l=26), (n=50, l=1), (n=50, l=12), (n=50, l=22), (n=50, l=28), (n=50, l=23), (n=50, l=27)}


In [59]:
dset1 = pycoll.datasets[78341]
ds1 = pd.Series(dset1.y, index=dset1.x)
ds1.index.name = 'E'
ds1.name = str(dset1.reaction)

dset2 = pycoll.datasets[78340]
ds2 = pd.Series(dset2.y, index=dset2.x)
ds2.index.name = 'E'
ds2.name = str(dset2.reaction)

In [60]:
df = pd.DataFrame((ds1, ds2)).T

In [61]:
df

,W+61 + H 1s → W+60 n=50;l=10 + H+,W+61 + H 1s → W+60 n=50;l=0 + H+
E,,
100000.0,2.082000e-20,0.000000e+00
500000.0,1.075000e-19,3.660000e-21
850000.0,1.096000e-20,3.169000e-22
1000000.0,4.756000e-21,4.311000e-23


In [114]:
# ... pre-process, get data_type
data_type = 'cross section'
x_units = 'eV'
process_types = ['HCX']

def aggregate_on(pycoll, species_formula):
    d = {}
    for ds in pycoll.datasets.values():
        for _, product_rp in ds.reaction.products:
            if product_rp.formula == species_formula:
                #k = ';'.join(str(state) for state in product_rp.states)
                k = tuple(product_rp.states)
                d[k] = pd.Series(ds.y, index=ds.x)
                d[k].index.name = 'E'
                print(product_rp)
    d = pd.DataFrame(d)
    d.sort_index(axis=1)
    d.name = data_type
    return d

In [115]:
d = aggregate_on(pycoll, Formula('W+60'))

W+60 n=50;l=0
W+60 n=50;l=10
W+60 n=50;l=11
W+60 n=50;l=12
W+60 n=50;l=13
W+60 n=50;l=14
W+60 n=50;l=15
W+60 n=50;l=16
W+60 n=50;l=17
W+60 n=50;l=18
W+60 n=50;l=19
W+60 n=50;l=1
W+60 n=50;l=20
W+60 n=50;l=21
W+60 n=50;l=22
W+60 n=50;l=23
W+60 n=50;l=24
W+60 n=50;l=25
W+60 n=50;l=26
W+60 n=50;l=27
W+60 n=50;l=28
W+60 n=50;l=29
W+60 n=50;l=2
W+60 n=50;l=30
W+60 n=50;l=31
W+60 n=50;l=32
W+60 n=50;l=33
W+60 n=50;l=34
W+60 n=50;l=35
W+60 n=50;l=36
W+60 n=50;l=37
W+60 n=50;l=38
W+60 n=50;l=3
W+60 n=50;l=4
W+60 n=50;l=5
W+60 n=50;l=6
W+60 n=50;l=7
W+60 n=50;l=8
W+60 n=50;l=9
W+60 n=50


In [116]:
type(d.columns[0][0])

pyvalem.states.key_value_pair.KeyValuePair

In [113]:
d

n=50                                            \
                    l=0          l=10          l=11          l=12   
E                                                                   
100000.0   0.000000e+00  2.082000e-20  3.449000e-20  3.747000e-20   
500000.0   3.660000e-21  1.075000e-19  1.163000e-19  1.251000e-19   
850000.0   3.169000e-22  1.096000e-20  1.132000e-20  1.215000e-20   
1000000.0  4.311000e-23  4.756000e-21  5.805000e-21  6.050000e-21   

                                                                   \
                   l=13          l=14          l=15          l=16   
E                                                                   
100000.0   5.293000e-20  9.754000e-20  1.190000e-19  1.647000e-19   
500000.0   1.435000e-19  1.518000e-19  1.687000e-19  1.892000e-19   
850000.0   1.547000e-20  1.591000e-20  1.709000e-20  2.005000e-20   
1000000.0  6.625000e-21  7.760000e-21  8.694000e-21  1.029000e-20   

                                       ...                              \
                   l=17          l=18  ...          l=37          l=38   
E                                      ...                               
100000.0   2.070000e-19  2.694000e-19  ...  6.393000e-19  5.947000e-22   
500000.0   2.028000e-19  2.281000e-19  ...  0.000000e+00  0.000000e+00   
850000.0   2.280000e-20  2.664000e-20  ...  0.000000e+00  0.000000e+00   
1000000.0  1.244000e-20  1.302000e-20  ...  0.000000e+00  0.000000e+00   

                                                                   \
                    l=3           l=4           l=5           l=6   
E                                                                   
100000.0   1.190000e-21  0.000000e+00  1.190000e-21  4.758000e-21   
500000.0   3.099000e-20  3.819000e-20  4.856000e-20  5.759000e-20   
850000.0   2.725000e-21  3.359000e-21  4.669000e-21  5.155000e-21   
1000000.0  1.308000e-21  1.710000e-21  2.271000e-21  2.860000e-21   

                                                                   
                    l=7           l=8           l=9           NaN  
E                                                                  
100000.0   8.326000e-21  7.732000e-21  1.011000e-20  4.319000e-17  
500000.0   7.558000e-20  8.351000e-20  8.846000e-20  6.645000e-18  
850000.0   7.268000e-21  7.162000e-21  8.430000e-21  4.083000e-19  
1000000.0  3.032000e-21  3.679000e-21  4.239000e-21  1.712000e-19  

[4 rows x 40 columns]

In [147]:
# ... pre-process, get data_type
data_type = 'cross section'
x_units = 'eV'
process_types = ['HCX']

def aggregate_on(pycoll, species_formula, ):
    d = {}
    for ds in pycoll.datasets.values():
        
        for _, product_rp in ds.reaction.products:
            if product_rp.formula == species_formula:
                #k = ';'.join(str(state) for state in product_rp.states)
                k = tuple((*([e[1] for e in ds.reaction.reactants]), *([e[1] for e in ds.reaction.products])))
                d[k] = pd.Series(ds.y, index=ds.x)
                d[k].index.name = 'E'
                print(product_rp)
    d = pd.DataFrame(d)
    d.sort_index(axis=1)
    d.name = data_type
    return d

In [148]:
d = aggregate_on(pycoll, Formula('W+60'))

W+60 n=50;l=0
W+60 n=50;l=10
W+60 n=50;l=11
W+60 n=50;l=12
W+60 n=50;l=13
W+60 n=50;l=14
W+60 n=50;l=15
W+60 n=50;l=16
W+60 n=50;l=17
W+60 n=50;l=18
W+60 n=50;l=19
W+60 n=50;l=1
W+60 n=50;l=20
W+60 n=50;l=21
W+60 n=50;l=22
W+60 n=50;l=23
W+60 n=50;l=24
W+60 n=50;l=25
W+60 n=50;l=26
W+60 n=50;l=27
W+60 n=50;l=28
W+60 n=50;l=29
W+60 n=50;l=2
W+60 n=50;l=30
W+60 n=50;l=31
W+60 n=50;l=32
W+60 n=50;l=33
W+60 n=50;l=34
W+60 n=50;l=35
W+60 n=50;l=36
W+60 n=50;l=37
W+60 n=50;l=38
W+60 n=50;l=3
W+60 n=50;l=4
W+60 n=50;l=5
W+60 n=50;l=6
W+60 n=50;l=7
W+60 n=50;l=8
W+60 n=50;l=9
W+60 n=50


In [149]:
d

W+61                                               \
                   H 1s                                                
          W+60 n=50;l=0 W+60 n=50;l=10 W+60 n=50;l=11 W+60 n=50;l=12   
                     H+             H+             H+             H+   
E                                                                      
100000.0   0.000000e+00   2.082000e-20   3.449000e-20   3.747000e-20   
500000.0   3.660000e-21   1.075000e-19   1.163000e-19   1.251000e-19   
850000.0   3.169000e-22   1.096000e-20   1.132000e-20   1.215000e-20   
1000000.0  4.311000e-23   4.756000e-21   5.805000e-21   6.050000e-21   

                                                                       \
                                                                        
          W+60 n=50;l=13 W+60 n=50;l=14 W+60 n=50;l=15 W+60 n=50;l=16   
                      H+             H+             H+             H+   
E                                                                       
100000.0    5.293000e-20   9.754000e-20   1.190000e-19   1.647000e-19   
500000.0    1.435000e-19   1.518000e-19   1.687000e-19   1.892000e-19   
850000.0    1.547000e-20   1.591000e-20   1.709000e-20   2.005000e-20   
1000000.0   6.625000e-21   7.760000e-21   8.694000e-21   1.029000e-20   

                                         ...                                \
                                         ...                                 
          W+60 n=50;l=17 W+60 n=50;l=18  ... W+60 n=50;l=37 W+60 n=50;l=38   
                      H+             H+  ...             H+             H+   
E                                        ...                                 
100000.0    2.070000e-19   2.694000e-19  ...   6.393000e-19   5.947000e-22   
500000.0    2.028000e-19   2.281000e-19  ...   0.000000e+00   0.000000e+00   
850000.0    2.280000e-20   2.664000e-20  ...   0.000000e+00   0.000000e+00   
1000000.0   1.244000e-20   1.302000e-20  ...   0.000000e+00   0.000000e+00   

                                                                   \
                                                                    
          W+60 n=50;l=3 W+60 n=50;l=4 W+60 n=50;l=5 W+60 n=50;l=6   
                     H+            H+            H+            H+   
E                                                                   
100000.0   1.190000e-21  0.000000e+00  1.190000e-21  4.758000e-21   
500000.0   3.099000e-20  3.819000e-20  4.856000e-20  5.759000e-20   
850000.0   2.725000e-21  3.359000e-21  4.669000e-21  5.155000e-21   
1000000.0  1.308000e-21  1.710000e-21  2.271000e-21  2.860000e-21   

                                                                   
                                                                   
          W+60 n=50;l=7 W+60 n=50;l=8 W+60 n=50;l=9     W+60 n=50  
                     H+            H+            H+            H+  
E                                                                  
100000.0   8.326000e-21  7.732000e-21  1.011000e-20  4.319000e-17  
500000.0   7.558000e-20  8.351000e-20  8.846000e-20  6.645000e-18  
850000.0   7.268000e-21  7.162000e-21  8.430000e-21  4.083000e-19  
1000000.0  3.032000e-21  3.679000e-21  4.239000e-21  1.712000e-19  

[4 rows x 40 columns]

In [146]:
d.columns

MultiIndex([(W+61, H 1s,  W+60 n=50;l=0, H+),
            (W+61, H 1s, W+60 n=50;l=10, H+),
            (W+61, H 1s, W+60 n=50;l=11, H+),
            (W+61, H 1s, W+60 n=50;l=12, H+),
            (W+61, H 1s, W+60 n=50;l=13, H+),
            (W+61, H 1s, W+60 n=50;l=14, H+),
            (W+61, H 1s, W+60 n=50;l=15, H+),
            (W+61, H 1s, W+60 n=50;l=16, H+),
            (W+61, H 1s, W+60 n=50;l=17, H+),
            (W+61, H 1s, W+60 n=50;l=18, H+),
            (W+61, H 1s, W+60 n=50;l=19, H+),
            (W+61, H 1s,  W+60 n=50;l=1, H+),
            (W+61, H 1s, W+60 n=50;l=20, H+),
            (W+61, H 1s, W+60 n=50;l=21, H+),
            (W+61, H 1s, W+60 n=50;l=22, H+),
            (W+61, H 1s, W+60 n=50;l=23, H+),
            (W+61, H 1s, W+60 n=50;l=24, H+),
            (W+61, H 1s, W+60 n=50;l=25, H+),
            (W+61, H 1s, W+60 n=50;l=26, H+),
            (W+61, H 1s, W+60 n=50;l=27, H+),
            (W+61, H 1s, W+60 n=50;l=28, H+),
            (W+61, H 1s, W+60 n=50

In [152]:
d.columns['W+61']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices